In [1]:
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import xgboost, string
import pandas as pd
import numpy as np

In [2]:
import multiprocessing
n_jobs_cnt = multiprocessing.cpu_count()-1
n_jobs_cnt

3

In [3]:
#importing review text and label files to dataframe

import pandas as pd

df_test_text = pd.read_table('imdb_test_text.txt', delim_whitespace=False, names=('A'))
df_test_labels = pd.read_table('imdb_test_labels.txt', delim_whitespace=False, names=('B'))

In [4]:
#merging them on index
df_test=df_test_text.join(df_test_labels, how='outer')
df_test.shape

(25000, 2)

In [5]:
df_train_text = pd.read_table('imdb_train_text.txt', delim_whitespace=False, names=('A'))
df_train_labels = pd.read_table('imdb_train_labels.txt', delim_whitespace=False, names=('B'))

In [6]:
#merging them on index
df_train=df_train_text.join(df_train_labels, how='outer')

In [56]:
# #changing column name
df_train=df_train.rename(columns={'A':'review_text',
                          'B':'review_label'})
df_train.index.name='S.No.'
df_train

In [57]:
#changing column name
df_test=df_test.rename(columns={'A':'review_text','B':'review_label'})
df_test.index.name='S.No.'
df_test

In [9]:
df_train.shape

(25000, 2)

In [11]:
df_all = pd.concat(objs=[df_train,
                         df_test],
                   axis=0)
df_all.reset_index(inplace=True)
df_all.shape

(50000, 3)

In [13]:
df_all.drop(labels=['S.No.'],
            inplace=True,
            axis=1)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(df_all['review_text'],
                                                    df_all['review_label'],
                                                    test_size=0.2,
                                                    random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40000,), (10000,), (40000,), (10000,))

In [17]:
# label encode the target variable
#mention a random split
encoder = LabelEncoder()
y_train_en = encoder.fit_transform(y_train)
y_test_en = encoder.transform(y_test)

y_train_en.shape, y_test_en.shape

((40000,), (10000,))

In [ ]:
y_test_en = pd.Series(y_test_en)
y_train_en = pd.Series(y_train_en)

In [55]:
i=500
while i < 4001:
    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=i, min_df=0.01, max_df=0.95)

    count_vect.fit(df_all['review_text'])

    # # transform the training and validation data using count vectorizer object
    xtrain_count =  count_vect.fit_transform(X_train)
    xvalid_count =  count_vect.transform(X_test)

    accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, y_train_en, xvalid_count)
    print("RF, Count Vectors for max features=",i,":",accuracy)
    i=i+500

RF, Count Vectors for max features= 500 : 0.3064
RF, Count Vectors for max features= 1000 : 0.3202
RF, Count Vectors for max features= 1500 : 0.3268
RF, Count Vectors for max features= 2000 : 0.327
RF, Count Vectors for max features= 2500 : 0.3277
RF, Count Vectors for max features= 3000 : 0.3346
RF, Count Vectors for max features= 3500 : 0.3191
RF, Count Vectors for max features= 4000 : 0.3239
Wall time: 10min 15s


In [47]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, y_test_en)